# Basic Example

This notebook follows the structure discussed in [https://luci-fitting.readthedocs.io/en/latest/example_basic.html](https://luci-fitting.readthedocs.io/en/latest/example_basic.html).

We will go through the steps necessary to use `LUCI`. As you will see, there are only a handful of commands required to use `LUCI`. This example is intentionally loquacious since it is, after all, an example :) 

In order to use `LUCI`, we need to import her, have data available to use, and have an initial guess. We have provided a command below to download a sample data cube that can be used to run the examples in this notebook.

#### * If you have not yet downloaded the sample data cube, uncomment the below cell and run and download the sample data cube.  
This will save a 900MB file called `sample_data.hdf5` to the **`ExampleData`** directory in the **`LUCI`** folder. Do not be concerned if it takes a few minutes to download.

In [1]:
# !wget -O Data/NGC628_SN3.hdf5  https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/2307000z.hdf5?RUNID=xc9le6u8llecp7fp

Let's start with importing `LUCI`. Just as described on the *readthedocs* page, we need to point our system path to our install location of the **`LUCI`** folder. 

We assume that the main project directory is the parent of the current Notebooks directory. For example, **`LUCI`**'s main directory lives in `/home/jessie/repos/LUCI` and my current directory is `/home/jessie/repos/LUCI/Examples`

However, we also provde instructions for how to point to the proper path if your main project directory is in a different directory. In this example, the data directory is `/home/carterrhea/Docments/NGC628`.

In [2]:
# Imports
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

# Get location of LUCI 
path = os.path.abspath(os.path.pardir)  
sys.path.insert(0, path)  # add LUCI to the available paths

from LuciBase import Luci
import LUCI.LuciPlotting as lplt

%config Completer.use_jedi=False  # enable autocompletion when typing in Jupyter notebooks

2022-09-07 07:32:25.548175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 07:32:25.548199: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Set the required parameters. We will be using our machine learning algorithm to get the initial guesses - this happens under the hood in `LuciFit`, so that the user is not required to think about the initial guess

In [3]:
# Initialize paths and set parameters
Luci_path = path + '/'
cube_dir = path + '/Data/ExampleData/'  # Path to data cube
cube_dir = '/mnt/carterrhea/carterrhea/NGC628'  # Full path to data cube (example 2)
cube_name = 'NGC628_SN3.merged.cm1.1.0'  # don't add .hdf5 extension
object_name = 'NGC628'
redshift = 0.000133  # Redshift of object
resolution = 1000 # The actual resolution is 400, but we don't have ML algorithms for that resolution, so use 1000

Intialize our LUCI object

In [4]:
cube = Luci(Luci_path, cube_dir + '/' + cube_name, cube_dir, object_name, redshift, resolution)

Reading in data...


100%|█████████████████████████████████████████████| 9/9 [00:11<00:00,  1.31s/it]


2022-09-07 07:32:40.280148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-07 07:32:40.280189: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-07 07:32:40.280215: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-09-07 07:32:40.280422: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create a deep frame

In [5]:
cube.create_deep_image()

Existing deep frame extracted from hdf5 file.


Let's extract a background region and take a look at it. The background region is defined in a ds9 region file in the `ExampleData` folder.

In [6]:
# We use 'mean = True' to take the mean of the emission in the region instead of the sum
bkg_axis, bkg_sky = cube.extract_spectrum_region(cube_dir+'/bkg.reg', mean=True)

100%|█████████████████████████████████████| 2064/2064 [00:00<00:00, 2743.71it/s]


We now fit part of our cube defined by the bounding box `500 < x < 1100` and `700 < y < 1300` with a Gaussian on the Halpha line, the NII-doublet, and the SII-doublet with a binning of 2. We will also tie all of our parameters together. 

For this example, we do not calculate the errors because it slows down calculations, but note that it can easily be done by adding the argument `uncertainty_bool=True`. If you want the full Bayesian calculation you can add `bayes_bool=True`.

In [ ]:
# Fit!
vel_map, broad_map, flux_map, chi2_fits = cube.fit_cube(
    ['Halpha', 'NII6548', 'NII6583', 'SII6716', 'SII6731'],  # lines
    'sincgauss',   # fit function
    [1,1,1,1,1],  # velocity relationship 
    [1,1,1,1,1],  # sigma relationship
    900, 1200,    # x bounds
    700, 1100,    # y bounds
    bkg=bkg_sky, binning=1,  
    n_threads=1
)

 14%|█████▋                                  | 57/400 [38:19<3:58:59, 41.81s/it]

Let's take a look at the flux map for Halpha. 

** Note that if the user enters 'flux' then the plotting method automatically takes the log base 10 and plots that. The flux is generally easier to visualize in log space.

In [ ]:
lplt.plot_map(flux_map[:,:,0], 'flux', cube_dir, cube.header, clims=[-18, -14])

Let's also take a look at the velocity and broadening maps.

In [ ]:
lplt.plot_map(vel_map[:,:,0], 'velocity', cube_dir, cube.header, clims=[0, 500])

In [ ]:
lplt.plot_map(broad_map[:,:,0], 'broadening', cube_dir, cube.header, clims=[0, 60])

And that is it! Congratulations, you have just used `LUCI`! 